In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import re
import time

# Function to parse WhatsApp chat data
def parse_whatsapp_chat(chat_file):
    with open(chat_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    messages = []
    for line in lines:
        match = re.match(r'(\d{1,2}/\d{1,2}/\d{2}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - (.*?): (.*)', line)
        if match:
            date, time, sender, message = match.groups()
            messages.append({
                'date': date,
                'time': time,
                'sender': sender,
                'message': message
            })
    return messages

# Load and parse chat file
chat_file = '/content/WhatsApp Chat with Dilluu.txt'
messages = parse_whatsapp_chat(chat_file)

# Filter friend's messages and create a dataset
friend_name = 'Dilluu'  # Replace with your friend's name as it appears in the chat
friend_messages = [msg['message'] for msg in messages if msg['sender'] == friend_name]
df = pd.DataFrame(friend_messages, columns=['text'])
dataset = Dataset.from_pandas(df)

# Load pre-trained model and tokenizer
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)

# Set pad_token_id to eos_token_id
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    inputs['labels'] = inputs['input_ids'].clone()
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Map:   0%|          | 0/5924 [00:00<?, ? examples/s]

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision training
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=500,
)

# Create Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Start training and measure time
start_time = time.time()
trainer.train()
end_time = time.time()

training_time = end_time - start_time
print(f"Training completed in {training_time / 60:.2f} minutes")


Step,Training Loss
500,0.375400
1000,0.186700
1500,0.178000


Step,Training Loss
500,0.375400
1000,0.186700
1500,0.178000
2000,0.142500
2500,0.135500
3000,0.136900
3500,0.112900
4000,0.107200


Training completed in 9.44 minutes


In [ ]:
import torch

In [ ]:

def generate_styled_response(prompt, model, tokenizer, max_length=150, temperature=0.7, top_k=50, top_p=0.9):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda') # Move inputs to GPU
    # print(inputs)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Truncate the prompt from the response
    # response = response[len(prompt):].strip()
    return response

# Example prompt

x = 0
while (x == 0):
  prompt = input("enter the prompttt: ")

  # Define generation parameters
  max_length = 400
  temperature = 0.1
  top_k = 50
  top_p = 0.3

  response = generate_styled_response(prompt, model, tokenizer, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p)
  print(response)
  x = int(input("enter 0 to coninue: "))


enter the prompttt: hi


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


hi chudhamm
enter 0 to coninue: 0
enter the prompttt: em chesthunnav
em chesthunnaviki vellam
enter 0 to coninue: 0
enter the prompttt: chaduthunnavaa sweetu
chaduthunnavaa sweetu
enter 0 to coninue: 0
enter the prompttt: good morningg
good morningg kaushuuu💗💗
enter 0 to coninue: 0
enter the prompttt: kaushuu 
kaushuu 
enter 0 to coninue: 0
enter the prompttt: sleeping i am
sleeping i amee
enter 0 to coninue: 0
enter the prompttt: byee ma
byee ma
enter 0 to coninue: 0
enter the prompttt: goood nightt
goood nightt😍😍


KeyboardInterrupt: Interrupted by user